In [1]:
import pygame
import numpy as np
import math
import random

class TankEnv:
    def __init__(self):
        pygame.init()
        self.width, self.height = 800, 600
        self.screen = pygame.display.set_mode((self.width, self.height))
        pygame.display.set_caption("Tank Reinforcement Learning Environment")
        self.clock = pygame.time.Clock()
        self.font = pygame.font.SysFont('Arial', 16)

        self.tank_speed = 3
        self.tank_size = 20
        self.target_radius = 15

        # Препятствия
        self.obstacles = [
            pygame.Rect(300, 200, 200, 50),
            pygame.Rect(400, 400, 100, 200),
            pygame.Rect(100, 300, 150, 30),
            pygame.Rect(600, 100, 50, 300)
        ]

        # Инициализация танка и цели
        self.reset()

    def reset(self):
        """Сброс среды в начальное состояние"""
        self.tank = {
            'x': random.randint(50, self.width-50),
            'y': random.randint(50, self.height-50),
            'angle': random.randint(0, 359),
            'health': 100
        }

        self.target = {
            'x': random.randint(50, self.width-50),
            'y': random.randint(50, self.height-50)
        }

        # Проверка, чтобы цель не появлялась в препятствиях
        target_rect = pygame.Rect(
            self.target['x'] - self.target_radius,
            self.target['y'] - self.target_radius,
            self.target_radius * 2,
            self.target_radius * 2
        )

        for obs in self.obstacles:
            if target_rect.colliderect(obs):
                return self.reset()  # Рекурсивно пробуем снова

        return self._get_state()

    def _get_state(self):
        """Получение текущего состояния среды"""
        dx = self.target['x'] - self.tank['x']
        dy = self.target['y'] - self.tank['y']
        distance = math.sqrt(dx**2 + dy**2)
        angle_to_target = math.atan2(dy, dx) - math.radians(self.tank['angle'])

        # Нормализация угла в диапазон [-π, π]
        angle_to_target = (angle_to_target + math.pi) % (2 * math.pi) - math.pi

        # Определение расстояний до препятствий в 4 направлениях
        obstacle_distances = self._get_obstacle_distances()

        return np.array([
            self.tank['x'] / self.width,  # Нормированная x-координата танка
            self.tank['y'] / self.height,  # Нормированная y-координата танка
            self.tank['angle'] / 360,     # Нормированный угол
            distance / math.sqrt(self.width**2 + self.height**2),  # Нормированное расстояние
            angle_to_target / math.pi,     # Нормированный угол до цели
            *obstacle_distances           # Расстояния до препятствий
        ], dtype=np.float32)

    def _get_obstacle_distances(self):
        """Вычисление расстояний до препятствий в 4 направлениях"""
        angles = [0, 90, 180, 270]  # Вперед, вправо, назад, влево
        distances = []

        for angle in angles:
            ray_angle = math.radians(self.tank['angle'] + angle)
            dist = self._cast_ray(ray_angle)
            distances.append(dist / max(self.width, self.height))  # Нормализация

        return distances

    def _cast_ray(self, angle):
        """Бросок луча для определения расстояния до препятствия"""
        step = 5
        x, y = self.tank['x'], self.tank['y']

        for d in range(0, 500, step):
            x += step * math.cos(angle)
            y += step * math.sin(angle)

            # Проверка выхода за границы
            if not (0 <= x <= self.width and 0 <= y <= self.height):
                return d

            # Проверка столкновения с препятствиями
            for obs in self.obstacles:
                if obs.collidepoint(x, y):
                    return d

        return 500  # Максимальное расстояние

    def step(self, action):
        """Выполнение действия и возврат нового состояния"""
        reward = -0.1  # Штраф за каждый шаг
        done = False
        info = {'reached_target': False, 'hit_obstacle': False}

        # Обработка действий
        if action == 0:  # Вперед
            self.tank['x'] += self.tank_speed * math.cos(math.radians(self.tank['angle']))
            self.tank['y'] += self.tank_speed * math.sin(math.radians(self.tank['angle']))
        elif action == 1:  # Назад
            self.tank['x'] -= self.tank_speed * math.cos(math.radians(self.tank['angle']))
            self.tank['y'] -= self.tank_speed * math.sin(math.radians(self.tank['angle']))
        elif action == 2:  # Влево
            self.tank['angle'] = (self.tank['angle'] - 5) % 360
        elif action == 3:  # Вправо
            self.tank['angle'] = (self.tank['angle'] + 5) % 360

        # Проверка границ
        self.tank['x'] = np.clip(self.tank['x'], 0, self.width)
        self.tank['y'] = np.clip(self.tank['y'], 0, self.height)

        # Проверка столкновений с препятствиями
        tank_rect = pygame.Rect(
            self.tank['x'] - self.tank_size//2,
            self.tank['y'] - self.tank_size//2,
            self.tank_size,
            self.tank_size
        )

        for obs in self.obstacles:
            if tank_rect.colliderect(obs):
                reward = -10
                done = True
                info['hit_obstacle'] = True
                break

        # Проверка достижения цели
        if math.dist((self.tank['x'], self.tank['y']),
                    (self.target['x'], self.target['y'])) < self.target_radius + self.tank_size//2:
            reward = 100
            done = True
            info['reached_target'] = True

        # Дополнительные награды/штрафы
        dx = self.target['x'] - self.tank['x']
        dy = self.target['y'] - self.tank['y']
        new_dist = math.sqrt(dx**2 + dy**2)

        if hasattr(self, 'prev_dist'):
            if new_dist < self.prev_dist:
                reward += 0.5  # Награда за приближение
            else:
                reward -= 0.3  # Штраф за удаление
        self.prev_dist = new_dist

        return self._get_state(), reward, done, info

    def render(self, episode=None, reward=None):
        """Отрисовка текущего состояния среды (исправленная версия)"""
        self.screen.fill((240, 240, 240))

        # Отрисовка препятствий
        for obs in self.obstacles:
            pygame.draw.rect(self.screen, (70, 70, 70), obs)

        # Отрисовка цели (исправленный вызов pygame.draw.circle)
        pygame.draw.circle(
            self.screen,
            (255, 50, 50),
            (int(self.target['x']), int(self.target['y'])),  # Центр как tuple (x,y)
            self.target_radius  # Радиус
        )

        # Отрисовка танка
        tank_color = (50, 150, 255) if self.tank['health'] > 50 else (255, 150, 50)
        tank_points = [
            (self.tank['x'] + self.tank_size*math.cos(math.radians(self.tank['angle']))),
            (self.tank['y'] + self.tank_size*math.sin(math.radians(self.tank['angle']))),
            (self.tank['x'] + (self.tank_size//2)*math.cos(math.radians(self.tank['angle']+120))),
            (self.tank['y'] + (self.tank_size//2)*math.sin(math.radians(self.tank['angle']+120))),
            (self.tank['x'] + (self.tank_size//2)*math.cos(math.radians(self.tank['angle']-120))),
            (self.tank['y'] + (self.tank_size//2)*math.sin(math.radians(self.tank['angle']-120)))
        ]
        pygame.draw.polygon(self.screen, tank_color, tank_points)

        # Отрисовка информации
        if episode is not None and reward is not None:
            info_text = f"Episode: {episode} | Reward: {reward:.1f} | Angle: {self.tank['angle']}°"
            text_surface = self.font.render(info_text, True, (0, 0, 0))
            self.screen.blit(text_surface, (10, 10))

        pygame.display.flip()
        self.clock.tick(60)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# State (пример для DQN)
state_size = 5  # [норм_x, норм_y, норм_угол, норм_расст, норм_угол_к_цели]

# Actions
action_space = [
    "вперед", "назад", "влево", "вправо", "огонь"
]

# Reward function (дополнение к коду среды)
def calculate_reward(self):
    reward = -1  # Штраф за шаг
    prev_dist = math.dist(self.prev_pos, (self.target['x'], self.target['y']))
    curr_dist = math.dist((self.tank['x'], self.tank['y']),
                         (self.target['x'], self.target['y']))

    if curr_dist < prev_dist:
        reward += 10  # Награда за приближение
    else:
        reward -= 5   # Штраф за удаление

    return reward

In [ ]:
import tensorflow as tf
from collections import deque
import random
import numpy as np

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=100000)
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.998
        self.batch_size = 128
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()
        self.loss_history = []

    def _build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, input_dim=self.state_size, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(self.action_size, activation='linear')
        ])
        model.compile(
            loss=tf.keras.losses.Huber(),
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.00025),
            metrics=['mae']
        )
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = np.reshape(state, [1, self.state_size])
        return np.argmax(self.model.predict(state, verbose=0)[0])

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        minibatch = random.sample(self.memory, self.batch_size)
        states = np.array([x[0] for x in minibatch])
        actions = np.array([x[1] for x in minibatch])
        rewards = np.array([x[2] for x in minibatch])
        next_states = np.array([x[3] for x in minibatch])
        dones = np.array([x[4] for x in minibatch])

        # Double DQN
        current_q = self.model.predict(states, verbose=0)
        next_q = self.model.predict(next_states, verbose=0)
        next_target_q = self.target_model.predict(next_states, verbose=0)

        max_actions = np.argmax(next_q, axis=1)
        targets = rewards + self.gamma * next_target_q[np.arange(self.batch_size), max_actions] * (1 - dones)

        current_q[np.arange(self.batch_size), actions] = targets

        history = self.model.fit(
            states,
            current_q,
            batch_size=self.batch_size,
            verbose=0
        )
        self.loss_history.append(history.history['loss'][0])

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Инициализация и обучение (20 эпизодов)
env = TankEnv()
agent = DQNAgent(state_size=9, action_size=5)
episodes = 10
target_update_freq = 2  # Обновлять target network каждые 10 эпизодов
reward_history = []

for e in range(episodes):
    state = env.reset()
    total_reward = 0
    while True:
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

        if done:
            reward_history.append(total_reward)
            if e % target_update_freq == 0:
                agent.update_target_model()

            print(f"Эпизод: {e+1:2d}/{episodes}, "
                  f"Награда: {total_reward:7.2f}, "
                  f"Epsilon: {agent.epsilon:.3f}, "
                  f"Loss: {np.mean(agent.loss_history[-10:] if agent.loss_history else 0):.4f}")
            break

    agent.replay()

# Вывод итоговой статистики
print("\nОбучение завершено!")
print(f"Средняя награда: {np.mean(reward_history):.2f}")
print(f"Максимальная награда: {max(reward_history):.2f}")
print(f"Минимальная награда: {min(reward_history):.2f}")

c:\Users\arau7\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Эпизод:  1/10, Награда: -7081.60, Epsilon: 1.000, Loss: 0.0000
Эпизод:  2/10, Награда:  -10.30, Epsilon: 0.998, Loss: 0.0139
Эпизод:  3/10, Награда: -3449.50, Epsilon: 0.996, Loss: 0.0135
Эпизод:  4/10, Награда:   -9.50, Epsilon: 0.994, Loss: 0.0132
Эпизод:  5/10, Награда: -5434.30, Epsilon: 0.992, Loss: 0.0133
Эпизод:  6/10, Награда:   -9.50, Epsilon: 0.990, Loss: 0.0133
Эпизод:  7/10, Награда:  -10.30, Epsilon: 0.988, Loss: 0.0133
